In [ ]:
import sys, os, natsort, glob

import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch, datasets
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F
from PIL import Image

import mediapipe as mp
import utils
import LantentNet
cudnn.enabled = True

gpu = 0 # GPU ID

#Initialize Model
model = LantentNet.LantentNet(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], 66)
saved_state_dict = torch.load('models\Latent_model_0,999.pkl') #For best yaw results
model.load_state_dict(saved_state_dict)




#Classification and Regression losses
criterion = nn.CrossEntropyLoss().cuda(gpu)
reg_criterion = nn.MSELoss().cuda(gpu)


MAE = torch.nn.L1Loss(reduction='mean')


softmax = nn.Softmax().cuda(gpu)

#For classification vector
idx_tensor = [idx for idx in range(66)]
idx_tensor = Variable(torch.FloatTensor(idx_tensor)).cuda(gpu)


model.cuda(gpu)
model.eval() #Inference mode

print("done")

# INFERENCE FOR INDIVIDUAL FACE IMAGES

In [ ]:
#input face image 
img = Image.open('wheel2.jpg')
frame = cv2.imread('wheel2.jpg')
frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

transformations = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

image = transformations(img)


image = image.view(1, 3, 224, 224) 
image = Variable(image).cuda(gpu)
x, yaw, pitch, roll = model(image)

yaw_predicted = softmax(yaw)#,dim = 1)
pitch_predicted = softmax(pitch)#,dim = 1)
roll_predicted = softmax(roll)#,dim =1)

yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 99
pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 99
roll_predicted = torch.sum(roll_predicted * idx_tensor, 1) * 3 - 99

yaw_predicted = yaw_predicted.view(1)
pitch_predicted = pitch_predicted.view(1)
roll_predicted = roll_predicted.view(1)

In [ ]:
import matplotlib.pyplot as plt
import utils

#Visualize result
utils.draw_axis(frame, yaw_predicted, pitch_predicted, roll_predicted, tdx = frame.shape[1] / 2, tdy= frame.shape[0] / 2, size = frame.shape[1]/2)

plt.figure()
plt.imshow(frame)

# INFERENCE FOR INDIVIDUAL IMAGES WITH FACE DETECTION

In [ ]:
import mediapipe as mp

mpFaceDetection = mp.solutions.face_detection
mpDraw = mp.solutions.drawing_utils
faceDetection = mpFaceDetection.FaceDetection(min_detection_confidence=0.5) #0.65


#input face image 
#img = Image.open('occlusion.jpg')
#frame = cv2.imread('occlusion.jpg')
img = Image.open('wheel2.jpg')
frame = cv2.imread('wheel2.jpg')
frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

results = faceDetection.process(frame)

detection = results.detections[0]

xmin = detection.location_data.relative_bounding_box.xmin
xmax = xmin + detection.location_data.relative_bounding_box.width
ymin = detection.location_data.relative_bounding_box.ymin
ymax = ymin + detection.location_data.relative_bounding_box.height

height = frame.shape[0]
width = frame.shape[1]

bbox_height = ymax-ymin

imgRGB = Image.fromarray(frame)

xmin_new = xmin*width 
ymin_new = ymin*height
xmax_new = xmax*width
ymax_new = ymax*height

# widen the face box margin for better pose estimation
xmin_new = xmin_new - 10
ymin_new = ymin_new - 20 
xmax_new = xmax_new + 10
ymax_new = ymax_new + 15

image = imgRGB.crop((xmin_new, ymin_new ,xmax_new, ymax_new))

transformations = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

image = transformations(img)


image = image.view(1, 3, 224, 224) 
image = Variable(image).cuda(gpu)
x, yaw, pitch, roll = model(image)

yaw_predicted = softmax(yaw)#,dim = 1)
pitch_predicted = softmax(pitch)#,dim = 1)
roll_predicted = softmax(roll)#,dim =1)

yaw_predicted = torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 99
pitch_predicted = torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 99
roll_predicted = torch.sum(roll_predicted * idx_tensor, 1) * 3 - 99

yaw_predicted = yaw_predicted.view(1)
pitch_predicted = pitch_predicted.view(1)
roll_predicted = roll_predicted.view(1)


In [ ]:
import matplotlib.pyplot as plt
import utils

#Visualize result
utils.draw_axis(frame, yaw_predicted, pitch_predicted, roll_predicted, tdx = frame.shape[1] / 2, tdy= frame.shape[0] / 2, size = frame.shape[1]/2)

plt.figure()
plt.imshow(frame)